https://coding-yoon.tistory.com/55

In [268]:
import numpy as np
import pandas as pd
import yfinance as yf

import torch
import torch.nn as nn
import torch.optim as optim

from matplotlib import pyplot as plt

In [269]:
MAX = 10
data = np.linspace(1, MAX, MAX, endpoint=True)
train_size = round(len(data) * .7)
train = data[:train_size]
test = data[train_size:]

In [270]:
def build_data(data, steps):
    print(data)
    x, y = [], []
    for i in range(steps, len(data)):
        x.append(data[i - steps:i])
        y.append(data[i:i + 1])
    return torch.FloatTensor(x), torch.FloatTensor(y)


debug = True
STEPS = 3
features = 1  # input_size
hidden_size = 1
x_train, y_train = build_data(train, STEPS)
x_train = torch.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
debug and print("x_train(batch_size, time_steps, features) = ", x_train.shape, x_train)
debug and print("y_train = ", y_train.shape, y_train)

[1. 2. 3. 4. 5. 6. 7.]
x_train(batch_size, time_steps, features) =  torch.Size([4, 3, 1]) tensor([[[1.],
         [2.],
         [3.]],

        [[2.],
         [3.],
         [4.]],

        [[3.],
         [4.],
         [5.]],

        [[4.],
         [5.],
         [6.]]])
y_train =  torch.Size([4, 1]) tensor([[4.],
        [5.],
        [6.],
        [7.]])


In [271]:
model = nn.RNN(features, hidden_size, batch_first=True)
with torch.no_grad():
    outputs, _status = model(x_train)
    print("Output =", outputs.shape, outputs, outputs[:, 2])

Output = torch.Size([4, 3, 1]) tensor([[[ 0.0184],
         [-0.6600],
         [-0.9073]],

        [[-0.6586],
         [-0.9074],
         [-0.9796]],

        [[-0.9215],
         [-0.9796],
         [-0.9958]],

        [[-0.9839],
         [-0.9958],
         [-0.9992]]]) tensor([[-0.9073],
        [-0.9796],
        [-0.9958],
        [-0.9992]])


In [274]:
criterion = nn.MSELoss()

lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
for epoch in range(10):
    outputs, _status = model(x_train)
    print(outputs[:, 2], y_train)
    loss = criterion(outputs[:, 2], y_train)

    optimizer.zero_grad()  #
    loss.backward()  # loss가 최소가 되게하는
    optimizer.step()  # 가중치 업

    print(f"Loss : {loss}")

tensor([[-0.8864],
        [-0.9734],
        [-0.9943],
        [-0.9988]], grad_fn=<SelectBackward0>) tensor([[4.],
        [5.],
        [6.],
        [7.]])
Loss : 43.11492156982422
tensor([[-0.8853],
        [-0.9730],
        [-0.9942],
        [-0.9988]], grad_fn=<SelectBackward0>) tensor([[4.],
        [5.],
        [6.],
        [7.]])
Loss : 43.11067581176758
tensor([[-0.8841],
        [-0.9727],
        [-0.9941],
        [-0.9988]], grad_fn=<SelectBackward0>) tensor([[4.],
        [5.],
        [6.],
        [7.]])
Loss : 43.10638427734375
tensor([[-0.8830],
        [-0.9723],
        [-0.9940],
        [-0.9987]], grad_fn=<SelectBackward0>) tensor([[4.],
        [5.],
        [6.],
        [7.]])
Loss : 43.10205078125
tensor([[-0.8818],
        [-0.9719],
        [-0.9939],
        [-0.9987]], grad_fn=<SelectBackward0>) tensor([[4.],
        [5.],
        [6.],
        [7.]])
Loss : 43.09767150878906
tensor([[-0.8806],
        [-0.9716],
        [-0.9938],
        [-0.9987